In [1]:
from tqdm import tqdm
from pathlib import Path
import json
import re
import numpy as np
import torchaudio
from IPython.display import Audio, display
import textgrids
import IPython.display as ipd
from PIL import Image
import matplotlib.pyplot as plt
import shutil

In [2]:
K = 5

In [3]:
aud_files = Path("../../../Datasets/flickr_audio")
save_dir = Path('../support_set')
save_dir.mkdir(parents=True, exist_ok=True)
image_base = Path('../../../Datasets/Flicker8k_Dataset')
image_save_dir = Path('../support_set/images')
image_save_dir.mkdir(parents=True, exist_ok=True)

In [4]:
vocab = []
with open('../data/test_keywords.txt', 'r') as f:
    for keyword in f:
        vocab.append(' '.join(keyword.split()))

In [5]:
alignments = {}
prev = ''
prev_wav = ''
prev_start = 0
with open(aud_files / 'flickr_8k.ctm', 'r') as f:
    for line in f:
        name, _, start, dur, label = line.strip().split()
        wav = name.split('.')[0] + '_' + name.split('#')[-1]
        label = label.lower()
        if label in vocab:
            if wav not in alignments: alignments[wav] = {}
            if label not in alignments[wav]: alignments[wav][label] = (float(start), float(start) + float(dur))
        prev = label
        prev_wav = wav
        prev_start = start

In [6]:
yoruba_alignments = {}
translation = {}
yoruba_vocab = []
with open(Path('../../../Datasets/yfacc_v6/Flickr8k_text/eng_yoruba_keywords.txt'), 'r') as f:
    for line in f:
        e, y = line.strip().split(', ')
        if e in vocab:
            translation[e] = y
            yoruba_vocab.append(y)

print(yoruba_vocab)

label_counts = {}
for txt_grid in Path('../../../Datasets/yfacc_v6/Flickr8k_alignment').rglob('*.TextGrid'):
    if str(txt_grid) == '../../../Datasets/yfacc_v6/Flickr8k_alignment/3187395715_f2940c2b72_0.TextGrid': continue
    grid = textgrids.TextGrid(txt_grid)
    wav = txt_grid.stem
    
    for interval in grid['words']:
        
        x = str(interval).split()
        label = str(interval).split('"')[1]
        start = x[-2].split('=')[-1]
        end = x[-1].split('=')[-1].split('>')[0]

        if label in yoruba_vocab:
            if wav not in yoruba_alignments: yoruba_alignments[wav] = {}
            if label not in yoruba_alignments[wav]: yoruba_alignments[wav][label] = (float(start), float(end)-float(start))
            if label not in label_counts: label_counts[label] = 0
            label_counts[label] += 1

['ọmọkùnrin', 'àwọn ajá', 'koríko', 'àpáta', 'omi']


In [7]:
support_set = np.load(save_dir / Path('support_set.npz'), allow_pickle=True)['support_set'].item()

In [13]:
for name in support_set:
    wav, im, yor, e_start, e_dur, word, y_start, y_end, y_word = support_set[name]
    print(wav, im, yor, e_start, e_dur, word, y_start, y_end, y_word)

    y_word = translation[word]
    y_dur = int((float(y_end)-float(y_start))*48000)
    y_offset = int(float(yoruba_alignments[name][y_word][0])*48000)
    y_aud, y_sr = torchaudio.load(Path('..') / yor, frame_offset=y_offset, num_frames=y_dur)
    play = y_aud.squeeze().numpy()
    ipd.display(ipd.Audio(play, rate=y_sr))
        
    break

#         impath = image_base / im 
#         img = Image.open(impath).convert('RGB')
#         plt.title(word)
#         plt.imshow(img)
#         plt.show()

#         ans = input(f'{count} / {K} {word}({y_word}): ')
#         if ans == 'y':

#             save_name = Path(wav).stem  + '_' + word + '.wav'
#             out_path = save_dir / Path(wav).parent.stem / Path(save_name)
#             out_path.parent.mkdir(parents=True, exist_ok=True)
#             torchaudio.save(out_path, aud, sr)

# #                 y_word = translation[word]
# #                 y_dur = int((float(yoruba_alignments[name][y_word][1])-float(yoruba_alignments[name][y_word][0]))*48000)
# #                 y_offset = int(float(yoruba_alignments[name][y_word][0])*48000)
# #                 aud, sr = torchaudio.load(Path('..') / yor, frame_offset=y_offset, num_frames=y_dur)

#             save_name = Path(yor).stem  + '_' + y_word + '.wav'
#             out_path = save_dir / Path('wavs') / Path(save_name)
#             out_path.parent.mkdir(parents=True, exist_ok=True)
# #                 play = aud.squeeze().numpy() 
# #                 ipd.display(ipd.Audio(play, rate=sr))
#             torchaudio.save(out_path, y_aud, y_sr)
    
#             out_path = image_save_dir / Path(impath.stem + '.jpg')
#             shutil.copyfile(impath, out_path)

#             support_set[name] = (wav, im, yor, alignments[name][word][0], alignments[name][word][1], word, yoruba_alignments[name][y_word][0], yoruba_alignments[name][y_word][1], y_word)

#             np.savez_compressed(
#                 save_dir / Path('support_set'), 
#                 support_set=support_set
#                 )
#             count += 1
#         if count == K: break

../../Datasets/flickr_audio/wavs/1096395242_fc69f0ae5a_0.wav ../../Datasets/Flicker8k_Dataset/1096395242_fc69f0ae5a.jpg ../../Datasets/yfacc_v6/flickr_audio_yoruba_test/S001_1096395242_fc69f0ae5a_0.wav 0.71 1.0699999999999998 boy 0.968911456680419 1.6018533798794035 ọmọkùnrin


In [ ]:
count = {}
for name in support_set:
    w = support_set[name][-1]
    if w not in count: count[w] = 0
    count[w] += 1
    
remove = []
for w in count:
    if count[w] != K:
        remove.append(w)
print(remove)
names = list(support_set.keys() )
for name in names:
    w = support_set[name][-1]
    if w in remove:
        del support_set[name]

In [ ]:
np.savez_compressed(
    save_dir / Path('support_set'), 
    support_set=support_set
    )